# VGG-16 Work

In [11]:
import os

# Path to the dataset folder
dataset_path = r'csv/raw-img'

# List all subdirectories (e.g., categories)
categories = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
print(f"Categories: {categories}")

# List files in each category
for category in categories:
    category_path = os.path.join(dataset_path, category)
    files = os.listdir(category_path)
    print(f"{category}: {len(files)} files")


Categories: ['cane', 'cavallo', 'elefante', 'farfalla', 'gallina', 'gatto', 'mucca', 'pecora', 'ragno', 'scoiattolo']
cane: 4863 files
cavallo: 2623 files
elefante: 1446 files
farfalla: 2112 files
gallina: 3098 files
gatto: 1668 files
mucca: 1866 files
pecora: 1820 files
ragno: 4821 files
scoiattolo: 1862 files


In [12]:
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel", "dog": "cane", "cavallo": "horse", "elephant" : "elefante", "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto", "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo"}